In [1]:
import QDMPy.data_loading as data_loading
import QDMPy.fit_interface as fit_interface
import QDMPy.fit_plots as fit_plots

import matplotlib
import matplotlib.pyplot as plt
import warnings

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

# Load the data in

In [3]:
options_dict = {
    # Argus: C:\\src\\QDMPY_test_image
    # Sam: /home/samsc/ResearchData/test_images/
    "base_dir": "/home/samsc/ResearchData/test_images/",
    "filepath": "ODMR - Pulsed_42",
    "custom_output_dir_prefix": None,
    "custom_output_dir_suffix": None,
    "custom_output_dir": None,
    
    "additional_bins": 8,

    "system_name": "Zyla",

    "ROI": "Full", 
    "ROI_start": [10, 10],
    "ROI_end": [200,200],
    
    "AOI_1_start": [60, 145],
    "AOI_1_end": [70, 155], 
    "AOI_2_start": [20, 20],
    "AOI_2_end": [24, 24], 
    "AOI_3_start": [30, 20],
    "AOI_3_end": [32, 24], 
    "single_pixel_check": [90, 150],

    "normalisation": "div",
    
    "fit_backend": "scipy",
    "fit_backend_comparison": ["scipy"],
    "fit_pixels": True, 
    "force_fit": False, 
    
    "use_ROI_avg_fit_res_for_all_pixels": True,
    
    "scipy_sub_threads": 2, 

    "fit_functions":  {"linear": 1, "lorentzian": 8},

#     "pos_guess": [2610, 2708, 2805, 2879, 2953, 3020, 3092, 3145],
    "pos_guess": [2631, 2723, 2787, 2866, 2955, 3022, 3066, 3130],
    "pos_range": 25,

    "amp_guess": -0.02,
    "amp_bounds": [-0.0300, -0.0003],

    "fwhm_guess": 9,
    "fwhm_bounds": [5, 20],

    "c_guess": 1,
    "c_bounds": [0.9995, 1.0005],

    "m_guess": 0,
    "m_bounds": [-1e-6, 1e-6],
    
    "scramble_pixels": True,
    
    "gpufit_tolerance": 1e-12,
    "gpufit_max_iterations": 25,
    "gpufit_estimator_id": "LSE"
}

In [4]:
# QDMPy_path = data_loading.DIR_PATH
# options = data_loading.load_options(path=QDMPy_path / "options/test_options_odmr.json", check_for_prev_result=True)
options = data_loading.load_options(options_dict=options_dict, check_for_prev_result=True)
fit_plots.set_mpl_rcparams(options)
raw_data, prelim_sweep_list = data_loading.load_raw_and_sweep(options)
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = data_loading.reshape_dataset(
        options, raw_data, prelim_sweep_list
    )
fig1 = fit_plots.plot_ROI_PL_image(options, PL_image)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
AOIs = data_loading.define_AOIs(options)
fig2 = fit_plots.plot_AOI_PL_images(options, PL_image_ROI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
aoi_spectra_plot = fit_plots.plot_AOI_spectra(options, AOIs, sig, ref, sweep_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit AOI, ROI averages, single pixel

In [7]:
fit_model = fit_interface.define_fit_model(options)
backend_ROI_results_lst = fit_interface.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)
fig3 = fit_plots.plot_ROI_avg_fits(options, backend_ROI_results_lst)

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/fit_scipy.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fit_result_collection_lst = fit_interface.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, AOIs, backend_ROI_results_lst)
AOI_spectra_fit_fig = fit_plots.plot_AOI_spectra_fit(options, sig, ref, sweep_list, AOIs, fit_result_collection_lst, fit_model)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit pixels

In [9]:
if not options["force_fit"] and options["found_prev_result"]:
    pixel_fit_params = fit_interface.load_prev_fit_results(options)
    warnings.warn("Plotting previous fit results.")
elif options["fit_pixels"]:
    pixel_fit_params = fit_interface.fit_pixels(options, sig_norm, sweep_list, fit_model, backend_ROI_results_lst[0])
else:
    pixel_fit_params = None # not fitting pixels, this stops plotting (e.g. via plot_param_images) from erroring

<ipython-input-9-390551cd98ba>:3: UserWarning: Plotting previous fit results.
  warnings.warn("Plotting previous fit results.")


# Plot (fit) parameters

In [10]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "c")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "m")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "pos")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "amp")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
data_loading.save_options(options)

In [16]:
options

{'base_dir': '/home/samsc/ResearchData/test_images/',
 'filepath': '/home/samsc/ResearchData/test_images/ODMR - Pulsed_42',
 'custom_output_dir_prefix': None,
 'custom_output_dir_suffix': None,
 'custom_output_dir': None,
 'raw_pixel_size': 2.44140625e-07,
 'ignore_ref': False,
 'additional_bins': 8,
 'old_binning_convention': False,
 'system_name': 'Zyla',
 'ROI': 'Full',
 'ROI_start': [10, 10],
 'ROI_end': [200, 200],
 'remove_start_sweep': 1,
 'remove_end_sweep': 0,
 'normalisation': 'div',
 'single_pixel_check': [90, 150],
 'AOI_1_start': (60, 145),
 'AOI_1_end': (70, 155),
 'AOI_2_start': (20, 20),
 'AOI_2_end': (24, 24),
 'AOI_3_start': (30, 20),
 'AOI_3_end': (32, 24),
 'AOI_4_start': None,
 'AOI_4_end': None,
 'AOI_5_start': None,
 'AOI_5_end': None,
 'fit_backend': 'scipy',
 'fit_backend_comparison': ['scipy'],
 'force_fit': False,
 'fit_pixels': True,
 'scramble_pixels': True,
 'use_ROI_avg_fit_res_for_all_pixels': True,
 'fit_functions': {'linear': 1, 'lorentzian': 8},
 'pos